In [3]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:90%;}
div.CodeMirror {font-family:Consolas; font-size:11pt;}
div.output {font-size:11pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:11pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
</style>
"""))

<font color="red" size="6"><b>ch09. 파일 i/o 프로그래밍</b></font>

# 1절. 파일에 데이터 저장하고 불러오기

In [4]:
# w 또는 wt : 기존의 파일이 있는 경우 덮어쓰기, 파일이 없는 경우 생성하여 쓰기
# a 또는 at : 기본의 파일이 있는 경우 append, 파일이 없는 경우 생성하여 쓰기
            # 쓰기 모드는 폴더가 없는 경우 예외 발생
# r 또는 rt : 기본의 파일을 읽는 경우, 파일이 없는 경우 예외 발생
f = open('data/ch09_sample.txt', 'w')
print(f.writable())

True


In [5]:
f.write('Hello\nWorld\nHello\nPython\n') # 한글을 write하면 CP949로 저장(win)
f.close()

In [6]:
# with구문을 사용하면 매번 close하는 코드가 불필요
with open('data/ch09_sample.txt', 'wt') as f:
    print('쓰기 가능한지 여부 :',f.writable())
    f.write('Python is easy.\nPython is smart.\n')

쓰기 가능한지 여부 : True


In [7]:
with open('data/ch09_sample.txt', 'at') as f:
    print('프린트 함수로 파일 쓰기가 가능합니다-콘솔 출력')
    print('프린트 함수로 파일 쓰기가 가능합니다-파일 출력', file=f)

프린트 함수로 파일 쓰기가 가능합니다-콘솔 출력


- euc-kr : 한글완성형 (믜x)
- cp949 : 확장된 한글완성형 (믜O) - 엑셀에서 
- utf-8 : 한글조합형(초중종성 따로 조합해서 저장) - 주피터노트북에서

In [8]:
with open('data/ch09_sample.txt', 'w', encoding='utf-8') as f:
    f.write('= 여러 줄의 text를 쓰기 =\n')
    f.write('1방법: Hello\nWorld\n')
    textlist = ['홍길동,30,서울\n', '김길동,20,안양\n']
    f.write('2방법 : \n')
    for line in textlist:
        f.write(line)
    f.write('3방법 :\n')
    f.writelines(textlist)

In [20]:
# 파일을 한줄 읽고 출력, 한줄 읽고 출력, ...
with open('data/ch09_sample.txt', 'r', encoding='utf-8') as f:
    line = f.readline() # 한줄 읽기
    while line != '':
        print(line, end='') # 파일에 \n이 포함되어서 출력시 개행은 제거
        line = f.readline()

= 여러 줄의 text를 쓰기 =
1방법: Hello
World
2방법 : 
홍길동,30,서울
김길동,20,안양
3방법 :
홍길동,30,서울
김길동,20,안양


In [9]:
# 파일을 한꺼번에 여러줄 읽어 list를 return
with open('data/ch09_sample.txt', 'rt', encoding='utf-8') as txtfile:
    lines = txtfile.readlines() # list로 return
    print(lines)

['= 여러 줄의 text를 쓰기 =\n', '1방법: Hello\n', 'World\n', '2방법 : \n', '홍길동,30,서울\n', '김길동,20,안양\n', '3방법 :\n', '홍길동,30,서울\n', '김길동,20,안양\n']


In [10]:
for line in lines:
    print(line, end='')

= 여러 줄의 text를 쓰기 =
1방법: Hello
World
2방법 : 
홍길동,30,서울
김길동,20,안양
3방법 :
홍길동,30,서울
김길동,20,안양


# 2절. 피클을 이용한 객체 저장 및 불러오기
## 2.1 형식이 있는 텍스트 데이터 불러오기

In [39]:
with open('data/ch09_member.txt','r',encoding='utf-8') as f:
    lines = f.readlines()
    # print(lines)
for line in lines:
    # print(line)
    data = line.strip().split(',')
    name = data[0]
    age = int(data[1].strip()) # strip은 앞뒤 white space 제거
    email = data[2].strip()
    address = data[3].strip()
    print('이름:{}, 나이:{}, 메일:{},\t주소:{}'.format(name, age, email, address))

이름:홍길동, 나이:20, 메일:kildong@hong.com,	주소:서울시 관악구
이름:홍길숙, 나이:25, 메일:kilsuk1@hong.com,	주소:서울시 영등포구
이름:신길동, 나이:30, 메일:shinkil@hong.com,	주소:서울시 동작구


In [40]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age  = age
        self.email= email
        self.address = address
    def __str__(self):
        return "이름:{}, 나이:{}, 메일:{},\t주소:{}".format(self.name,
                                            self.age, self.email, self.address)
    def __eq__(self, other):
        '''
        user1.__eq__(user2) 호출시 user1의 속성변수들과 user2의 속성변수들이 모두 같은 여부
        '''
        if isinstance(other, Member):
            return self.name==other.name and self.age == other.age \
                    and self.email==other.email and self.address==other.address
        else:
            return False
    def as_dict(self): # 객체를 딕셔너리로 바꿔 return
        return {'name' : self.name,
                'age' : self.age,
                'email':self.email,
                'address':self.address}

In [41]:
user = Member('홍길동',22, 'hong@hong.com', '서울') # 생성자 test
print(user) # __str__ test
print(user.as_dict()) # as_dict test

이름:홍길동, 나이:22, 메일:hong@hong.com,	주소:서울
{'name': '홍길동', 'age': 22, 'email': 'hong@hong.com', 'address': '서울'}


In [42]:
# 파일의 내용을 member list나 딕셔너리 list로 읽기
user_list = [] # member list
user_dict = [] # 딕셔너리 list
with open('data/ch09_member.txt','r', encoding='utf-8') as f:
    lines = f.readlines()
for line in lines:
    data = line.strip().split(',')
    name = data[0]
    age = int(data[1].strip())
    email = data[2].strip()
    address = data[3].strip()
    user = Member(name, age, email, address)
    user_list.append(user)
    user_dict.append(user.as_dict())

In [43]:
# user_list (객체 list)
for user in user_list:
    print(user)

이름:홍길동, 나이:20, 메일:kildong@hong.com,	주소:서울시 관악구
이름:홍길숙, 나이:25, 메일:kilsuk1@hong.com,	주소:서울시 영등포구
이름:신길동, 나이:30, 메일:shinkil@hong.com,	주소:서울시 동작구


In [44]:
# user_dict (딕셔너리 list)
user_dict

[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울시 관악구'},
 {'name': '홍길숙',
  'age': 25,
  'email': 'kilsuk1@hong.com',
  'address': '서울시 영등포구'},
 {'name': '신길동', 'age': 30, 'email': 'shinkil@hong.com', 'address': '서울시 동작구'}]

In [45]:
user1 = Member("홍길동", 20, 'h@h.com', '서울')
user2 = Member("홍길동", 20, 'h@h.com', '서울')

In [46]:
user1 == user2

True

In [47]:
print(user1)

이름:홍길동, 나이:20, 메일:h@h.com,	주소:서울


In [48]:
user1 = Member("홍길동", 20, 'h@h.com', '서울')
user2 = Member("김길동", 25, 'k@h.com', '인천')
user3 = Member("신길동", 30, 's@h.com', '포천')
user_list = [user1, user2, user3]

In [25]:
for user in user_list:
    print(user)

이름:홍길동, 나이:20, 메일:h@h.com,	주소:서울
이름:김길동, 나이:25, 메일:k@h.com,	주소:인천
이름:신길동, 나이:30, 메일:s@h.com,	주소:포천


In [49]:
# 피클링을 이용한 객체를 파일에 저장하기
import pickle
with open('data/ch09_member.data', 'wb') as f:
    pickle.dump(user_list, f)

In [50]:
del user1, user2, user3

In [51]:
# 피클링을 이용한 파일에서 객체 데이터 읽기
with open('data/ch09_member.data', 'rb') as f:
    load_user_list = pickle.load(f)

In [52]:
for user in load_user_list:
    print(user)

이름:홍길동, 나이:20, 메일:h@h.com,	주소:서울
이름:김길동, 나이:25, 메일:k@h.com,	주소:인천
이름:신길동, 나이:30, 메일:s@h.com,	주소:포천


In [53]:
# user_list와 load_user_list가 모두 같은지 여부
result = []
for idx in range(len(user_list)):
    result.append(user_list[idx] == load_user_list[idx])
result

[True, True, True]

In [54]:
all(result) # 모든 요ㅗ들이 Ture인지 여부

True

# 3절. csv 파일 읽기/쓰기

### 3.1 reader

In [57]:
#utf-8로 인코딩된 csv파일 읽기
import csv
with open('data/ch09_member1.csv', 'r', encoding='UTF-8') as f:
    reader = csv.reader(f)
#    print(list(reader))
    for row in reader:
        print(row)

['홍길동', ' 20', ' "killdong@hong.com"', ' "서울시 관악구"']
['김길동', ' 40', ' "kimdong@hong.com"', ' "서울시 영등포구"']
['신길동', ' 30', ' "sindong@hong.com"', ' "서울시 동작구"']


In [58]:
# CP949로 인코딩 된 csv파일 read
with open('data/ch09_member1-cp.csv') as f:
    reader = csv.reader(f)
    result = list(reader)
print(result)

[['홍길동', ' 20', ' "killdong@hong.com"', ' "서울시 관악구"'], ['김길동', ' 40', ' "kimdong@hong.com"', ' "서울시 영등포구"'], ['신길동', ' 30', ' "sindong@hong.com"', ' "서울시 동작구"']]


In [63]:
# csv 파일의 내용을 딕셔너리 리스트로 가져오기
user_dict = []
with open('data/ch09_member1.csv', encoding='utf-8') as f:
    reader = csv.reader(f, quotechar = '"',
                       quoting=csv.QUOTE_NONNUMERIC)
    for row in reader:
        user_dict.append({'name' : row[0],
                         'age' : row[1],
                        'email' : row[2],
                        'address' : row[3]})
print('파일에서 읽어온 딕셔너리 리스트')
print(user_dict)

파일에서 읽어온 딕셔너리 리스트
[{'name': '홍길동', 'age': 20.0, 'email': 'killdong@hong.com', 'address': '서울시 관악구'}, {'name': '김길동', 'age': 40.0, 'email': 'kimdong@hong.com', 'address': '서울시 영등포구'}, {'name': '신길동', 'age': 30.0, 'email': 'sindong@hong.com', 'address': '서울시 동작구'}]


#### 3.2 writer

In [64]:
user_list = [['홍길동',20,'hong@hong.com', '서울'],
            ['신길동',30,'shin@hong.com','인천'],]

In [68]:
# newline = '' 옵션을 빼면 개행이 2번
try:
    with open('data/ch09_member2-write-cp.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        for user in user_list:
            writer.writerow(user)
except FileNotFoundError as e:
    print(e)

In [70]:
with open('data/ch09_member2-write.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)
    writer.writerows(user_list)

## 3.3 DictReader
- csv 파일의 자료를 딕셔너리 데이터로 읽기

In [72]:
# header가 있는 csv파일을 딕셔너리로 읽기
with open('data/ch09_member3.csv', 'r', encoding='utf-8') as f:
    dict_reader = csv.DictReader(f)
#    print(list(dict_reader))
for row in dict_reader:
    if row['job'] is None:
        print(row['name'], row['age'], row['address'])

ValueError: I/O operation on closed file.

In [73]:
# cf. csv파일을 데이터 프레임으로
import pandas as pd
member = pd.read_csv('data/ch09_member3.csv') # encoding='utf-8'기본
member

,name,age,email,address,job
0,홍길동,20,h@h.com,서울시 관악구,NaN
1,김길동,40,k@h.com,서울시 영등포구,팀장
2,신길동,30,s@h.com,서울시 동작구,NaN


In [74]:
type(member)

pandas.core.frame.DataFrame

In [77]:
# header가 없는 csv파일을 딕셔너리로 읽기
with open('data/ch09_member1-cp.csv', 'r', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, fieldnames=['Name', 'Age','Email', 'Address'])
    for row in dict_reader:
        print(row['Name'], row['Age'], row['Email'], row['Address'])

홍길동  20  "killdong@hong.com"  "서울시 관악구"
김길동  40  "kimdong@hong.com"  "서울시 영등포구"
신길동  30  "sindong@hong.com"  "서울시 동작구"


In [78]:
with open('data/ch09_member1-cp.csv', 'r', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, fieldnames=['Name', 'Age','Email', 'Address', 'Job'])
    for row in dict_reader:
        print(row['Name'], row['Age'], row['Email'], row['Address'], row['Job'])

홍길동  20  "killdong@hong.com"  "서울시 관악구" None
김길동  40  "kimdong@hong.com"  "서울시 영등포구" None
신길동  30  "sindong@hong.com"  "서울시 동작구" None


In [95]:
# header가 없는 csv파일을 딕셔너리로 읽기
with open('data/ch09_member1-cp.csv', 'r', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, fieldnames=['Name','Age','Email','Address'],
                                restkey='Job')
    for row in dict_reader:
#         print(row)
        if row['Job'][0] == '':
            print(row['Name'], row['Age'], row['Email'], row['Address'])
        else:
            print(row['Name'], row['Age'], row['Email'], row['Address'], row['Job'][0])

홍길동 20 killdong@hong.com 서울시 관악구
김길동 40 kimdong@hong.com 서울시 영등포구 팀장
신길동 30 sindong@hong.com 서울시 동작구


In [102]:
# ch09_member1-cp.csv의 내용을 딕셔너리 list로 받아보세요
userlist = [] # 딕셔너리 list
# 파일의 내용을 userlist에 담기
with open('data/ch09_member1-cp.csv','r', encoding='cp949') as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        userlist.append(row)
print(userlist)

[{'홍길동': '김길동', '20': '40', 'killdong@hong.com': 'kimdong@hong.com', '서울시 관악구': '서울시 영등포구', '': '팀장'}, {'홍길동': '신길동', '20': '30', 'killdong@hong.com': 'sindong@hong.com', '서울시 관악구': '서울시 동작구', '': ''}]


## 3.4 DictWriter
- 딕셔너리 리스트를 csv파일로 쓰기

In [103]:
user1 = {'name':'홍길동','age':20,'emaeil':'h@hong.com','address':'서울'}
user2 = {'name':'신길동','age':25,'emaeil':'s@hong.com','address':'부산'}
user3 = {'name':'김길동','age':35,'emaeil':'k@hong.com','address':'인천'}
user_list = [user1, user2, user3]
filenames = list(user1.keys())
print(filenames)

['name', 'age', 'emaeil', 'address']


In [117]:
with open('data/ch09_member4.csv', 'a', encoding='UTF-8', newline='') as f:
    dict_writer = csv.DictWriter(f,fieldnames=filenames)
#     dict_writer.writeheader() # header 한 줄 쓰기
#     for user in user_list:
#         dict_writer.writerow(user)
    dict_writer.writerows(user_list)

# 4절. JSON파일에 저장하고 불러오기 (Dump, Load)

## 4.1 JSON dump(파일에 저장)